<a href="https://colab.research.google.com/github/Mhizta-Mammut/Connectivity-based-Outlier-Factor/blob/master/COF_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary packages
## this is warnings module which is use in ignoring the warnings of the program #
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
 
 
## Data manupulations
import scipy, scipy.ndimage
from skimage.util import random_noise
from PIL import Image
import numpy as np
from numpy.lib.stride_tricks import as_strided
from tqdm import tnrange, tqdm_notebook
from math import sqrt, floor

In [ ]:
# This code will accept noise density as input from the user
noise_density = int(input('Please enter the noise density of the system in percentage: '))

Please enter the noise density of the system in percentage: 95


In [ ]:
class C_O_F(object):
	"""This is the class that consist all utility functions that helps in calculating C_O_F"""
	def __init__(self, image):
		super(C_O_F, self).__init__()

		self.img = image
		self.gaussian_noise = self.noisy('gaussian', self.img)
		self.salt_and_pepper_noise = self.noisy('s&p', self.img)
	
		self.image = self.noisy('s&p', self.img)


	def getImage(self):
		"""
		Get the Pixel Ratio of the given image... i.e (256 x 256 pixels)
		"""
		return self.image.shape

	def getGaussianNoise(self):
		"""
		Get the image with gaussian noise applied in it
		"""
		return self.gaussian_noise


	def getSaltPepperNoise(self):
		"""
		Get the image with gaussian noise applied in it
		"""
		return self.salt_and_pepper_noise

	def sliding_window(self, arr, window_size):
		""" Construct a sliding window view of the array"""
		arr = np.asarray(arr)
		window_size = int(window_size)
		if arr.ndim != 2:
				raise ValueError("need 2-D input")
		if not (window_size > 0):
				raise ValueError("need a positive window size")
		shape = (arr.shape[0] - window_size + 1,
							arr.shape[1] - window_size + 1,
							window_size, window_size)
		if shape[0] <= 0:
				shape = (1, shape[1], arr.shape[0], shape[3])
		if shape[1] <= 0:
				shape = (shape[0], 1, shape[2], arr.shape[1])
		strides = (arr.shape[1]*arr.itemsize, arr.itemsize,
								arr.shape[1]*arr.itemsize, arr.itemsize)
		return as_strided(arr, shape=shape, strides=strides)

	def _neighbors(self, i, j, d = 1):
		"""Return d-th neighbors of cell (i, j)"""
		arr = self.image
		w = self.sliding_window(arr, 2*d+1)

		ix = np.clip(i - d, 0, w.shape[0]-1)
		jx = np.clip(j - d, 0, w.shape[1]-1)

		i0 = max(0, i - d - ix)
		j0 = max(0, j - d - jx)
		i1 = w.shape[2] - max(0, d - i + ix)
		j1 = w.shape[3] - max(0, d - j + jx)

		return w[ix, jx][i0:i1,j0:j1].ravel()


	def cof(self):
		"""
		This function will calculate the COF
		"""
		img = np.array(self.image)

		rows, cols = img.shape
		
		return [[self.cal_cof(i, j) for i in range(rows)] for j in tqdm_notebook(range(cols), desc="Calculating COF...")]

	
# ################################################################################################################################################################

	def noisy(self, noise_typ, img):
		"""
			Noise: This function will apply the impulse noise to the given image.
			Parameter:
        ----------
            noise_typ : Type of Noise to apply...i.e gaussian, salt-and-pepper.
            img : The coordinates of the given Matrix
		"""
		image = np.array(img)
		density = noise_density/100

		if noise_typ == "gaussian":
			row,col = image.shape
			mean = 0
					#var = 0.1
					#sigma = var**0.5
			gauss = np.random.normal(mean, density, (row,col))
			gauss = gauss.reshape(row,col)
			noisy = image + gauss
			return noisy
		elif noise_typ == "s&p":
			row,col = image.shape
			s_vs_p = density
			amount = 0.004
			out = image
					# Salt mode
			num_salt = np.ceil(amount * image.size * s_vs_p)
			coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
			out[coords] = 1

					# Pepper mode
			num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
			coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
			out[coords] = 0
			return out
		else:
			raise ValueError("Noise unknown; you can only use either 'gauss' or 's&p', '{}' given".format(noise_typ))
   
	 
	def noise_and_nonNoise(self, original_image, noisy_image):
		"""
			noise_and_nonNoise: This function will 
			Parameter:
        ----------
            noise_typ : Type of Noise to apply...i.e gaussian, salt-and-pepper.
            imaege : The coordinates of the given Matrix
		"""
		rows, cols = original_image.shape
		original_pixels = []
		noisy_pixels = []

		# map = []
		for i in range(rows):
			for j in range(cols):
				if float(original_image[i][j]) == np.round(noisy_image[i][i]):
					original_pixels.append(0)
				else:
					noisy_pixels.append(1)
		 			# map[i][j] = 0

		return original_pixels, noisy_pixels

	def generateBinaryMap(self):
		"""
			generateBinaryMap: This function will generate binary map of the given noisy image 
													(0 for original pixels, 1 for noisy pixels)
		"""
		original_image = np.array(self.image)
		noisy_image = self.gaussian_noise
		rows, cols = original_image.shape

		return [[0 if float(original_image[i][j]) == np.round(noisy_image[i][i]) else 1 for i in range(rows)] for j in tqdm_notebook(range(cols), desc="Generating Binary Map...")]

	def comparePixels(self):
		"""
		comparePixels: This function will compare the pixels between two images
		"""
		gauss_noise = self.noise_and_nonNoise(self.image, self.gaussian_noise)
		sp_noise = self.noise_and_nonNoise(self.image, self.salt_and_pepper_noise)

		gauss_original_pixels, gauss_noisy_pixels = gauss_noise
		sp_original_pixels, sp_noisy_pixels = sp_noise

		return (len(gauss_original_pixels), len(gauss_noisy_pixels)), (len(sp_original_pixels), len(sp_noisy_pixels))

	def ratioOfPixels(self):
		"""
		ratioOfPixels: This function will return the ratio of noisy pixels and original pixels
		"""
		pixels = self.comparePixels()

		gauss, sp = pixels

		gauss_original_ratio = ((gauss[0]/(gauss[0]+gauss[1])) * 100)
		gauss_noisy_ratio = ((gauss[1]/(gauss[0]+gauss[1])) * 100)
	
		sp_original_ratio = ((sp[0]/(sp[0]+sp[1])) * 100)
		sp_noisy_ratio = ((sp[1]/(sp[0]+sp[1])) * 100)

		return (gauss_original_ratio, gauss_noisy_ratio), (sp_original_ratio, sp_noisy_ratio)
	
	def cof_value_to_binary(self, value):
		"""
		This function will convert the calculated COF value to Binary Value...i.e 0 = original, 1 = noisy
		"""
		value = np.array(value)
		rows, cols = value.shape
		
		cof_v = [[int(round(value[i][j])) for i in range(rows)] for j in range(cols)]

		cof_binary = [[1 if cof_v[i][j] >= 1 else 0 for i in range(rows)] for j in range(cols)]

		return cof_binary

	# Calculating COF value
	def cal_cof(self, i, j):
		# standard deviations
		knn = self._neighbors(i, j)
		sd = round(np.std(knn), 2)
		n = len(knn)
		k = n - 1
		
		c = 0
		sigma = 0
		s = []
		for i in range(n):
			sigma = abs(knn[i]- sd)
	 		
			c = floor((2* abs((k + 1) - sigma))/(k * (k + 1)))
			s.append(c)
		t = sum(s)/n
		return round(t, 2)
	
	def detects(self):
		"""
		Produce final result
		"""
		cof_ = self.cof()
		cof_value = np.array(self.cof_value_to_binary(cof_))
		binary_value = np.array(self.generateBinaryMap())
		rows, cols = binary_value.shape

		return [['TP' if binary_value[i][j] == 1 and cof_value[i][j] == 1 else 'FP' if binary_value[i][j] == 1 and cof_value[i][j] == 0 else 'FN' if binary_value[i][j] == 0 and cof_value[i][j] == 1 else 'TN' if binary_value[i][j] == 0 and cof_value[i][j] == 0 else 11 for i in range(rows)] for j in tqdm_notebook(range(cols), desc="Detecting noisy pixels...")]



In [ ]:
img = [[50, 50, 67, 52, 51], 
        [62, 61, 47, 46, 182], 
        [51, 63, 51, 181, 182],
        [51, 55, 182, 180, 183],
        [66, 150, 181, 179, 183]]

imgs = Image.open('caman.tif')
print(np.array(imgs))

cof_object = C_O_F(imgs)

tp = []
tn = []
fp = []
fn = []
pixels = cof_object.detects()
pixels = np.array(pixels)
rows, cols = pixels.shape
for i in range(rows):
	for j in range(cols):
		if pixels[i][j] == 'TP':
			tp.append(pixels[i][j])
		elif pixels[i][j] == 'TN':
			tn.append(pixels[i][j])
		elif pixels[i][j] == 'FP':
			fp.append(pixels[i][j])
		elif pixels[i][j] == 'FN':
			fn.append(pixels[i][j])
		else:
			print(None)
	 
print('\n\n')
print("True Positive => {}".format(len(tp)))
print("True Negative => {}".format(len(tn)))
print("False Positive => {}".format(len(fp)))
print("False Negative => {}".format(len(fn)))

[[154 157 157 ... 150 154 155]
 [152 150 155 ... 155 153 148]
 [157 153 156 ... 154 158 151]
 ...
 [114 132 123 ... 135 137 114]
 [121 126 130 ... 133 130 113]
 [  0   0   0 ...   0   0   0]]






True Positive => 49786
True Negative => 612
False Positive => 14051
False Negative => 1087
